<a href="https://colab.research.google.com/github/VladislavVolkovS/CNN-Cifar100flops-0.7e6/blob/main/%D0%A1%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном задании необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6). Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%. Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Импортируем библиотеки

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

Глобальные константы

In [3]:
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

Загружаем модель

In [4]:
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

169001437/169001437 [==============================] - 565s 3us/step


 Преобразуем метки классов в one_hot формат

In [5]:
y_train = (y_train == np.arange(CLASSES)).astype(np.float32)
y_val = (y_val == np.arange(CLASSES)).astype(np.float32)

Убедимся, что ячейка выполняется без ошибок

In [6]:
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

Зададим архитектуру модели

In [131]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(8, (2, 2), activation='selu',
                  input_shape=(32, 32, 3), padding='same', strides = 2),
    tf.keras.layers.Dense(100, activation='selu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.SeparableConv2D(4, (2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='selu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50, activation='selu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(CLASSES, activation='softmax')
])

Вычислим количество операций

In [132]:
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6533e6


Информация о модели

In [133]:
model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 16, 16, 8)         104       
                                                                 
 dense_121 (Dense)           (None, 16, 16, 100)       900       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 8, 8, 100)        0         
 g2D)                                                            
                                                                 
 dropout_83 (Dropout)        (None, 8, 8, 100)         0         
                                                                 
 separable_conv2d_27 (Separa  (None, 7, 7, 4)          804       
 bleConv2D)                                                      
                                                                 
 flatten_37 (Flatten)        (None, 196)             

Ячейка для изменения(получения более высокой точности)

In [134]:
model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
            ),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

Обучение модели

In [135]:
model.fit(
            x=X_train,
            y=y_train,
            validation_data=(X_val, y_val),
            batch_size=BATCH_SIZE,
            callbacks=[
                tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),           
            ],
            use_multiprocessing=True,
            workers=8,
            epochs=256
        )

Epoch 1/256
391/391 [==============================] - 21s 52ms/step - loss: 3.7886 - accuracy: 0.1157 - val_loss: 3.5610 - val_accuracy: 0.1644
Epoch 2/256
391/391 [==============================] - 20s 52ms/step - loss: 3.3018 - accuracy: 0.1969 - val_loss: 3.2479 - val_accuracy: 0.2141
Epoch 3/256
391/391 [==============================] - 20s 52ms/step - loss: 3.1485 - accuracy: 0.2253 - val_loss: 3.0878 - val_accuracy: 0.2414
Epoch 4/256
391/391 [==============================] - 20s 51ms/step - loss: 3.0617 - accuracy: 0.2432 - val_loss: 2.9761 - val_accuracy: 0.2660
Epoch 5/256
391/391 [==============================] - 22s 55ms/step - loss: 2.9905 - accuracy: 0.2550 - val_loss: 2.9239 - val_accuracy: 0.2779
Epoch 6/256
391/391 [==============================] - 20s 51ms/step - loss: 2.9511 - accuracy: 0.2620 - val_loss: 2.8963 - val_accuracy: 0.2878
Epoch 7/256
391/391 [==============================] - 20s 51ms/step - loss: 2.9080 - accuracy: 0.2714 - val_loss: 2.8697 - val_ac